# Install Library

In [ ]:
!pip install scikit-learn pandas matplotlib nltk wordcloud joblib
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

# Import Library

In [ ]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Load Dataset

In [ ]:
df = pd.read_csv('/content/sample_data/ujaran_kebencian_binary.csv')
df.head()

# Distribusi Label

In [ ]:
df['binary_label'].value_counts().plot(kind='bar'); plt.show()

# Preprocessing

In [ ]:
stop_eng = set(stopwords.words('english'))

def clean_text(text):
    if not isinstance(text, str): return ''
    text = text.lower()
    text = re.sub(r'http\S+|www\.\S+', ' ', text)
    text = re.sub(r'rt\b', ' ', text)
    text = re.sub(r'@[A-Za-z0-9_]+', ' ', text)
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stop_eng and len(t)>1]
    return ' '.join(tokens)

df['clean'] = df['Tweet'].apply(clean_text)
df.head()

# TF-IDF

In [ ]:
tfidf = TfidfVectorizer(max_features=10000)
X = tfidf.fit_transform(df['clean'])
y = df['binary_label']
print(X.shape)

# Split Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train Model

In [ ]:
model = LinearSVC()
model.fit(X_train, y_train)

# Evaluation

In [ ]:
y_pred = model.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# Wordcloud Label 1

In [ ]:
texts_label1 = ' '.join(df[df['binary_label']==1]['clean'])
wc = WordCloud(width=800, height=400).generate(texts_label1)
plt.imshow(wc); plt.axis('off'); plt.show()